In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Circuit
from EC04 import EC04
import pandas as pd
from numpy import random

In [2]:
frequencies = Circuit.dec_step(1e-2, 1e5, 72)

In [11]:
import random
# from openpyxl import Workbook
samples=50000
# Generate random values for three variables
R1 = [random.uniform(15,35) for _ in range(samples)]
CPE_Y1 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n1 = [random.uniform(0.5, 1) for _ in range(samples)]

In [12]:
output_array = np.stack((R1, CPE_Y1, CPE_n1), axis=1)

In [13]:
num_rows = len(output_array)
num_rows

50000

In [22]:
num_freq = len(output_array)
num_rows = 72
num_columns = len(output_array[0])

# Create an empty numpy array to store the frequency response data
frequency_response_array = np.empty((num_freq, num_rows, num_columns))
# len(frequency_response_array[0][0])

3

In [25]:
# As output_array is a 2D NumPy array
for idx in range(output_array.shape[0]):
    row = output_array[idx]
    b = EC04(*row)
    Z = b.freq_response(frequencies)

    # Store the frequency values  in the first column
    frequency_response_array[idx, :, 0] = frequencies

    # Store the Absolute Value of Z in the second column
    frequency_response_array[idx, :, 1] = np.abs(Z)

    # Store the Phase values of Z in the third column
    frequency_response_array[idx, :, 2] = np.angle(Z)


# Now, frequency_response_array has the shape (50000, 72, 3)



In [36]:
# As frequency_response_array has shape (50000, 72, 3) and output_array has shape (50000, 3)
# Creating an empty array to store the mapped data
mapped_data = np.empty_like(output_array)

# Iterate over each element of both arrays simultaneously
for idx, (freq_resp_element, output_element) in enumerate(zip(frequency_response_array, output_array)):
    # Map the each element of frequency_response_array to the corresponding element of output_array
   
    # We reshape freq_resp_element to (1, 72, 3) to match the dimensions of output_element
    mapped_data[idx] = freq_resp_element[0].reshape(1, -1)  # Reshape to (1, 3)

# Now, mapped_data contains the mapped elements where each element has shape (1, 3)


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assuming frequency_response_array has shape (50000, 72, 3) and output_array has shape (50000, 3)

# Reshape frequency_response_array to have shape (50000, 216)
X = frequency_response_array.reshape((frequency_response_array.shape[0], -1))

y = output_array

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Training R^2 Score:", train_score)
print("Testing R^2 Score:", test_score)


Training R^2 Score: 0.9971164890118592
Testing R^2 Score: 0.9968494304250194


In [46]:
for i in range(1000):    
    random_idx = np.random.randint(0, len(frequency_response_array))
    # Select the random element from frequency_response_array
    random_element = frequency_response_array[random_idx]

    # Reshape the random element to match the input shape expected by the model
    reshaped_element = random_element.reshape(1, -1)

    # Predict the corresponding output value
    predicted_output = model.predict(reshaped_element)

    # Extract the corresponding actual output value from the output array
    actual_output = output_array[random_idx]

    # Print the predicted and actual output values for comparison
    print("Predicted Output:", predicted_output)
    print("Actual Output:", actual_output)


Predicted Output: [[2.16019478e+01 8.02096678e-04 8.48927247e-01]]
Actual Output: [2.16006557e+01 7.87285993e-04 8.48927455e-01]
Predicted Output: [[3.40336227e+01 8.84539913e-05 9.97391781e-01]]
Actual Output: [3.39792260e+01 5.05591212e-05 9.97388552e-01]
Predicted Output: [[2.86771488e+01 2.03094212e-04 6.75942073e-01]]
Actual Output: [2.86772004e+01 2.06980944e-04 6.75942075e-01]
Predicted Output: [[1.94004135e+01 1.53840287e-04 7.18723588e-01]]
Actual Output: [1.94025814e+01 1.31492119e-04 7.18723680e-01]
Predicted Output: [[2.80670776e+01 9.90785193e-05 9.15585746e-01]]
Actual Output: [2.80678719e+01 8.91526348e-05 9.15585511e-01]
Predicted Output: [[1.66693649e+01 5.54243801e-04 7.64928290e-01]]
Actual Output: [1.66688107e+01 5.61758478e-04 7.64928260e-01]
Predicted Output: [[2.95471497e+01 1.72198517e-04 7.84860275e-01]]
Actual Output: [2.95471123e+01 1.82971438e-04 7.84860301e-01]
Predicted Output: [[2.02926140e+01 2.42040260e-04 8.01193387e-01]]
Actual Output: [2.02925225e+01

In [49]:
from sklearn.metrics import mean_squared_error,accuracy_score

# Predict outputs for the entire test set
predicted_outputs = model.predict(X_test)

# Compute mean squared error
mse = mean_squared_error(y_test, predicted_outputs)
accuracy=100-mse
# Print the mean squared error
print("Mean Squared Error:", mse)
print("Accuracy:",accuracy)


Mean Squared Error: 0.0007326396675926178
Accuracy: 99.99926736033241
